<a href="https://colab.research.google.com/github/suleiman-odeh/NLP_Project_Team16/blob/main/Gemma_2/zero_shot_indirect_Gemma_2_9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.4 MB/s eta 0:00:00


In [ ]:
"""
This cell is for loading the model
I have run it already but cleared the output since it cant be upload to github
"""
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login

# logging using user access token
login()

#  Define 4-Bit Configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,              # Loading in 4-bit
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, # Compute in 16-bit for speed, store in 4-bit
)

# Load model
model_id = "google/gemma-2-9b-it"

print(f"Loading {model_id} in 4-bit...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("Model loaded successfully!")


In [10]:
"""
This cell is for loading the data and mapping the evasion labels to clarity IDs.
Also its for defining the prompt function based on the paper.
"""
import pandas as pd

# Load the data
try:
    df = pd.read_json('QEvasion_cleaned.jsonl', lines=True)
    print(f"Data loaded. Total rows: {len(df)}")
except FileNotFoundError:
    print("ERROR: Upload 'QEvasion_cleaned.jsonl' first.")

# Maps the 9 evasion labels to the 3 clarity IDs (0, 1, 2)
EVASION_TO_CLARITY_ID = {
    'Explicit': 0,
    'Implicit': 1, 'Dodging': 1, 'Deflection': 1, 'General': 1, 'Partial/half-answer': 1,
    'Declining to answer': 2, 'Claims ignorance': 2, 'Clarification': 2
}

# prompt function based on the paper
def create_gemma_indirect_prompt(question, answer):
    system_instruction = """Based on a segment of the interview in which the interviewer poses a series of questions,
    classify the type of response provided by the interviewee for the following question using the following taxonomy
    and then provide a chain of thought explanation for your decision:

<Taxonomy>
1. Explicit: The information requested is explicitly stated (in the requested form).
2. Implicit: The information requested is given, but without being explicitly stated (not in the expected form).
3. General: The information provided is too general/lacks the requested specificity.
4. Partial/half-answer: Offers only a specific component of the requested information.
5. Dodging: Ignoring the question altogether.
6. Deflection: Starts on topic but shifts the focus and makes a different point than what is asked.
7. Declining to answer: Acknowledges the question but directly or indirectly refusing to answer at the moment.
8. Claims ignorance: The answerer claims/admits not to know the answer themselves.
9. Clarification: Does not provide the requested information and asks for clarification.
You are required to respond with a single term corresponding to the Taxonomy code and only.


### Part of the interview ###
"""

    # Gemma-Specific Chat Template
    prompt = f"""<start_of_turn>user
{system_instruction}
Question: "{question}"
Answer: "{answer}"

### Question ###
"{question}"
<end_of_turn>
<start_of_turn>model
Taxonomy code:
"""
    return prompt

print("Setup Complete.")

Data loaded. Total rows: 3756
Setup Complete.


In [11]:
"""
This cell is for running the zero shot on the test data
"""
import torch
import gc
import re
from tqdm import tqdm

# take test data
test_df = df[df['split_type'] == 'test'].copy()
print(f"Processing Test Set: {len(test_df)} samples")

predictions_evasion = []
predictions_clarity = []
raw_outputs = []

# Inference Loop
print("Starting Zero-Shot Inference ...")
model.eval()

for index, row in tqdm(test_df.iterrows(), total=len(test_df)):
    # Create Prompt
    prompt_text = create_gemma_indirect_prompt(row['question'], row['cleaned_answer'])

    # Tokenize
    inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")

    # Generate
    # more tokens allowed because of COT
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode
    generated_text = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    clean_text = generated_text.strip()
    raw_outputs.append(clean_text)

    # since the output might be a large text, we search, which word appeared from the evasion list in the text
    detected_label = "Error"

    found_labels = []
    for valid_label in EVASION_TO_CLARITY_ID.keys():
        # # If multiple labels found, pick the first one mentioned.
        if re.search(r'\b' + re.escape(valid_label) + r'\b', clean_text, re.IGNORECASE):
            found_labels.append(valid_label)

    if found_labels:
        best_match = min(found_labels, key=lambda l: clean_text.lower().find(l.lower()))
        detected_label = best_match
    else:
        detected_label = "Error"

    # Map to Clarity ID
    mapped_id = EVASION_TO_CLARITY_ID.get(detected_label, -1)

    predictions_evasion.append(detected_label)
    predictions_clarity.append(mapped_id)

# store results in the dataset for the evaultion in the next cell
test_df['raw_output'] = raw_outputs
test_df['pred_evasion'] = predictions_evasion
test_df['pred_clarity_id'] = predictions_clarity

print("Results are stored in the 'test_df' variable.")


Processing Test Set: 308 samples
Starting Zero-Shot Inference ...


100%|██████████| 308/308 [40:18<00:00,  7.85s/it]

Results are stored in the 'test_df' variable.


In [33]:
"""
Our results contained from the zeroshot included listing numbers and, sometimes, listing text. So, at the beginning,
we need to convert the numbers into their corresponding evasion_label so that we can evaluate them.
"""
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import re
import numpy as np

# ==========================================
# DEFINE THE MAPS
# ==========================================
CODE_TO_ID = {
    '1': 5, # Explicit
    '2': 7, # Implicit
    '3': 6, # General
    '4': 8, # Partial/half-answer
    '5': 4, # Dodging
    '6': 3, # Deflection
    '7': 2, # Declining to answer
    '8': 0, # Claims ignorance
    '9': 1  # Clarification
}

# ID -> Text Label
ID_TO_LABEL = {
    5: 'Explicit',
    7: 'Implicit',
    6: 'General',
    8: 'Partial/half-answer',
    4: 'Dodging',
    3: 'Deflection',
    2: 'Declining to answer',
    0: 'Claims ignorance',
    1: 'Clarification'
}

# Text -> ID
WORD_TO_ID = {
    'explicit': 5,
    'implicit': 7,
    'general': 6,
    'partial': 8, 'half': 8,
    'dodging': 4,
    'deflection': 3,
    'declining': 2,
    'ignorance': 0, 'claims': 0,
    'clarification': 1
}

# Clarity mapping
def get_clarity_id(d_id):
    if d_id == 5: return 0  # Explicit
    if d_id in [7, 4, 3, 6, 8]: return 1 # Implicit, Dodging, Deflection, General, Partial
    if d_id in [2, 0, 1]: return 2 # Declining, Ignorance, Clarification
    return -1

CLARITY_LABELS = {0: 'Clear Reply', 1: 'Ambivalent', 2: 'Clear Non-Reply'}


# ==========================================
# create new colimns
# ==========================================
def parse_model_output(row):
    text = str(row['raw_output']).strip().lower()

    # 1. Try Code (e.g. "6")
    match = re.match(r'^(\d)\b', text)
    if match:
        return CODE_TO_ID.get(match.group(1), -1)

    # 2. Try Word keywords
    for word, id_num in WORD_TO_ID.items():
        if word in text:
            return id_num

    return -1

print("Processing predictions...")
test_df['pred_evasion_id'] = test_df.apply(parse_model_output, axis=1)

# getting strings
test_df['pred_evasion'] = test_df['pred_evasion_id'].map(ID_TO_LABEL)

# Generate Clarity
test_df['pred_clarity_id'] = test_df['pred_evasion_id'].apply(get_clarity_id)
test_df['pred_clarity'] = test_df['pred_clarity_id'].map(CLARITY_LABELS)

# Filter valid rows
valid_df = test_df[test_df['pred_evasion_id'] != -1].copy()
print(f"Total Samples: {len(test_df)}")
print(f"Valid Samples: {len(valid_df)}")
print("-" * 60)


# ==========================================
# EVASION EVALUATION
# ==========================================
print("TASK 2: EVASION (9-Class)")
print("="*60)

def get_ground_truth(row):
    pred = row['pred_evasion_id']
    humans = [row['annotator1_id'], row['annotator2_id'], row['annotator3_id']]

    valid_humans = []
    for h in humans:
        try:
            if pd.notna(h): valid_humans.append(int(h))
        except: pass

    if pred in valid_humans:
        return pred
    else:
        return valid_humans[0] if valid_humans else -1

# Create Ground Truth
valid_df['true_evasion_id'] = valid_df.apply(get_ground_truth, axis=1)

# Filter for scoring
final_df = valid_df[valid_df['true_evasion_id'] != -1].copy()

y_true = final_df['true_evasion_id']
y_pred = final_df['pred_evasion_id']

# --- Metrics ---
acc = accuracy_score(y_true, y_pred)

# Macro
prec_macro = precision_score(y_true, y_pred, average='macro', zero_division=0)
rec_macro = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1_macro = f1_score(y_true, y_pred, average='macro')

# Weighted
prec_weighted = precision_score(y_true, y_pred, average='weighted', zero_division=0)
rec_weighted = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1_weighted = f1_score(y_true, y_pred, average='weighted')

# --- Print ---
print(f"Accuracy:           {acc:.4f}")
print("-" * 30)
print(f"Macro Precision:    {prec_macro:.4f}")
print(f"Macro Recall:       {rec_macro:.4f}")
print(f"Macro F1:           {f1_macro:.4f}")
print("-" * 30)
print(f"Weighted Precision: {prec_weighted:.4f}")
print(f"Weighted Recall:    {rec_weighted:.4f}")
print(f"Weighted F1:        {f1_weighted:.4f}")
print("-" * 60)

# Target Names
target_names = [
    "Claims ignorance (0)",
    "Clarification (1)",
    "Declining to answer (2)",
    "Deflection (3)",
    "Dodging (4)",
    "Explicit (5)",
    "General (6)",
    "Implicit (7)",
    "Partial/half-answer (8)"
]

print(classification_report(y_true, y_pred, labels=range(9), target_names=target_names))

Processing predictions...
Total Samples: 308
Valid Samples: 308
------------------------------------------------------------
TASK 2: EVASION (9-Class) PERFORMANCE
Accuracy:           0.3312
------------------------------
Macro Precision:    0.5337
Macro Recall:       0.3021
Macro F1:           0.3076
------------------------------
Weighted Precision: 0.5183
Weighted Recall:    0.3312
Weighted F1:        0.3179
------------------------------------------------------------
                         precision    recall  f1-score   support

   Claims ignorance (0)       0.00      0.00      0.00         9
      Clarification (1)       1.00      0.50      0.67         4
Declining to answer (2)       1.00      0.20      0.33        10
         Deflection (3)       0.21      0.93      0.34        43
            Dodging (4)       0.57      0.08      0.14        51
           Explicit (5)       0.61      0.28      0.39        95
            General (6)       0.71      0.19      0.30        26
    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [31]:
# Filter
correct_df = test_df[test_df['pred_evasion'] != "Error"].copy()

print(f"Showing 6 samples of SUCCESSFUL extractions (Total: {len(correct_df)})")
print("-" * 80)

# Display raw output vs extracted label
cols_to_show = ['raw_output', 'pred_evasion']
print(correct_df[cols_to_show].head(6))

Showing 6 samples of SUCCESSFUL extractions (Total: 308)
--------------------------------------------------------------------------------
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                raw_output  \
3448                                                                                                                                                                                                                                                                            

In [36]:
"""
save results for comparison
"""
import pandas as pd


comparison_df = test_df[['pred_evasion', 'annotator1', 'annotator2', 'annotator3']].copy()

comparison_df.to_csv("predictions_comparison.csv", index=True)

print("Saved 'predictions_comparison.csv' (Contains: Index, Prediction, 3 Annotators)")


test_df.to_csv("full_test_dataset_zs_indirect.csv", index=True)

print("Saved 'full_test_dataset_zs_indirect.csv'")

Saved 'predictions_comparison.csv' (Contains: Index, Prediction, 3 Annotators)
Saved 'full_test_dataset_zs_indirect.csv'
